# [`re`](https://docs.python.org/3/library/re.html#module-re "re: Regular expression operations.")&nbsp;— Regular expression operations[](https://docs.python.org/3/library/re.html#module-re "Permalink to this headline")

**Source code:**&nbsp;[Lib/re.py](https://github.com/python/cpython/tree/3.8/Lib/re.py)

---

This module provides regular expression matching operations similar to those found in Perl.

Both patterns and strings to be searched can be Unicode strings ([`str`](https://docs.python.org/3/library/stdtypes.html#str "str")) as well as 8-bit strings ([`bytes`](https://docs.python.org/3/library/stdtypes.html#bytes "bytes")). However, Unicode strings and 8-bit strings cannot be mixed: that is, you cannot match a Unicode string with a byte pattern or vice-versa; similarly, when asking for a substitution, the replacement string must be of the same type as both the pattern and the search string.

Regular expressions use the backslash character (`'\'`) to indicate special forms or to allow special characters to be used without invoking their special meaning. This collides with Python’s usage of the same character for the same purpose in string literals; for example, to match a literal backslash, one might have to write&nbsp;`'\\\\'`&nbsp;as the pattern string, because the regular expression must be&nbsp;`\\`, and each backslash must be expressed as&nbsp;`\\`&nbsp;inside a regular Python string literal. Also, please note that any invalid escape sequences in Python’s usage of the backslash in string literals now generate a&nbsp;[`DeprecationWarning`](https://docs.python.org/3/library/exceptions.html#DeprecationWarning "DeprecationWarning")&nbsp;and in the future this will become a&nbsp;[`SyntaxError`](https://docs.python.org/3/library/exceptions.html#SyntaxError "SyntaxError"). This behaviour will happen even if it is a valid escape sequence for a regular expression.

The solution is to use Python’s raw string notation for regular expression patterns; backslashes are not handled in any special way in a string literal prefixed with&nbsp;`'r'`. So&nbsp;`r"\n"`&nbsp;is a two-character string containing&nbsp;`'\'`&nbsp;and&nbsp;`'n'`, while&nbsp;`"\n"`&nbsp;is a one-character string containing a newline. Usually patterns will be expressed in Python code using this raw string notation.

It is important to note that most regular expression operations are available as module-level functions and methods on&nbsp;[compiled regular expressions](https://docs.python.org/3/library/re.html#re-objects). The functions are shortcuts that don’t require you to compile a regex object first, but miss some fine-tuning parameters.

See also

&nbsp;

The third-party&nbsp;[regex](https://pypi.org/project/regex/)&nbsp;module, which has an API compatible with the standard library&nbsp;[`re`](https://docs.python.org/3/library/re.html#module-re "re: Regular expression operations.")&nbsp;module, but offers additional functionality and a more thorough Unicode support.

## Regular Expression Syntax[](https://docs.python.org/3/library/re.html#regular-expression-syntax "Permalink to this headline")

A regular expression (or RE) specifies a set of strings that matches it; the functions in this module let you check if a particular string matches a given regular expression (or if a given regular expression matches a particular string, which comes down to the same thing).

Regular expressions can be concatenated to form new regular expressions; if&nbsp;_A_&nbsp;and&nbsp;_B_&nbsp;are both regular expressions, then&nbsp;_AB_&nbsp;is also a regular expression. In general, if a string&nbsp;_p_&nbsp;matches&nbsp;_A_&nbsp;and another string&nbsp;_q_matches&nbsp;_B_, the string&nbsp;_pq_&nbsp;will match AB. This holds unless&nbsp;_A_&nbsp;or&nbsp;_B_&nbsp;contain low precedence operations; boundary conditions between&nbsp;_A_&nbsp;and&nbsp;_B_; or have numbered group references. Thus, complex expressions can easily be constructed from simpler primitive expressions like the ones described here. For details of the theory and implementation of regular expressions, consult the Friedl book&nbsp;[[Frie09]](https://docs.python.org/3/library/re.html#frie09), or almost any textbook about compiler construction.

A brief explanation of the format of regular expressions follows. For further information and a gentler presentation, consult the&nbsp;[Regular Expression HOWTO](https://docs.python.org/3/howto/regex.html#regex-howto).

Regular expressions can contain both special and ordinary characters. Most ordinary characters, like&nbsp;`'A'`,&nbsp;`'a'`, or&nbsp;`'0'`, are the simplest regular expressions; they simply match themselves. You can concatenate ordinary characters, so&nbsp;`last`&nbsp;matches the string&nbsp;`'last'`. (In the rest of this section, we’ll write RE’s in&nbsp;`this special style`, usually without quotes, and strings to be matched&nbsp;`'in single quotes'`.)

Some characters, like&nbsp;`'|'`&nbsp;or&nbsp;`'('`, are special. Special characters either stand for classes of ordinary characters, or affect how the regular expressions around them are interpreted.

Repetition qualifiers (`*`,&nbsp;`+`,&nbsp;`?`,&nbsp;`{m,n}`, etc) cannot be directly nested. This avoids ambiguity with the non-greedy modifier suffix&nbsp;`?`, and with other modifiers in other implementations. To apply a second repetition to an inner repetition, parentheses may be used. For example, the expression&nbsp;`(?:a{6})*`&nbsp;matches any multiple of six&nbsp;`'a'`characters.

The special characters are:

`.`
:    (Dot.) In the default mode, this matches any character except a newline. If the&nbsp;[`DOTALL`](https://docs.python.org/3/library/re.html#re.DOTALL "re.DOTALL")&nbsp;flag has been specified, this matches any character including a newline.

`^`
:    (Caret.) Matches the start of the string, and in&nbsp;[`MULTILINE`](https://docs.python.org/3/library/re.html#re.MULTILINE "re.MULTILINE")&nbsp;mode also matches immediately after each newline.

`$`
:    Matches the end of the string or just before the newline at the end of the string, and in&nbsp;[`MULTILINE`](https://docs.python.org/3/library/re.html#re.MULTILINE "re.MULTILINE")&nbsp;mode also matches before a newline.&nbsp;`foo`&nbsp;matches both ‘foo’ and ‘foobar’, while the regular expression&nbsp;`foo$`&nbsp;matches only ‘foo’. More interestingly, searching for&nbsp;`foo.$`&nbsp;in&nbsp;`'foo1\nfoo2\n'`&nbsp;matches ‘foo2’ normally, but ‘foo1’ in&nbsp;[`MULTILINE`](https://docs.python.org/3/library/re.html#re.MULTILINE "re.MULTILINE")&nbsp;mode; searching for a single&nbsp;`$`&nbsp;in&nbsp;`'foo\n'`&nbsp;will find two (empty) matches: one just before the newline, and one at the end of the string.

`*`
:    Causes the resulting RE to match 0 or more repetitions of the preceding RE, as many repetitions as are possible.&nbsp;`ab*`&nbsp;will match ‘a’, ‘ab’, or ‘a’ followed by any number of ‘b’s.

`+`
:    Causes the resulting RE to match 1 or more repetitions of the preceding RE.&nbsp;`ab+`&nbsp;will match ‘a’ followed by any non-zero number of ‘b’s; it will not match just ‘a’.

`?`
:    Causes the resulting RE to match 0 or 1 repetitions of the preceding RE.&nbsp;`ab?`&nbsp;will match either ‘a’ or ‘ab’.

`*?`,&nbsp;`+?`,&nbsp;`??`
:    The&nbsp;`'*'`,&nbsp;`'+'`, and&nbsp;`'?'`&nbsp;qualifiers are all&nbsp;_greedy_; they match as much text as possible. Sometimes this behaviour isn’t desired; if the RE&nbsp;`<.*>`&nbsp;is matched against&nbsp;`'<a> b <c>'`, it will match the entire string, and not just&nbsp;`'<a>'`. Adding&nbsp;`?`&nbsp;after the qualifier makes it perform the match in&nbsp;_non-greedy_&nbsp;or&nbsp;_minimal_&nbsp;fashion; as&nbsp;_few_characters as possible will be matched. Using the RE&nbsp;`<.*?>`&nbsp;will match only&nbsp;`'<a>'`.

`{m}`
:    Specifies that exactly&nbsp;_m_&nbsp;copies of the previous RE should be matched; fewer matches cause the entire RE not to match. For example,&nbsp;`a{6}`&nbsp;will match exactly six&nbsp;`'a'`&nbsp;characters, but not five.

`{m,n}`
:    Causes the resulting RE to match from&nbsp;_m_&nbsp;to&nbsp;_n_&nbsp;repetitions of the preceding RE, attempting to match as many repetitions as possible. For example,&nbsp;`a{3,5}`&nbsp;will match from 3 to 5&nbsp;`'a'`&nbsp;characters. Omitting&nbsp;_m_&nbsp;specifies a lower bound of zero, and omitting&nbsp;_n_&nbsp;specifies an infinite upper bound. As an example,&nbsp;`a{4,}b`&nbsp;will match&nbsp;`'aaaab'`&nbsp;or a thousand&nbsp;`'a'`&nbsp;characters followed by a&nbsp;`'b'`, but not&nbsp;`'aaab'`. The comma may not be omitted or the modifier would be confused with the previously described form.

`{m,n}?`
:    Causes the resulting RE to match from&nbsp;_m_&nbsp;to&nbsp;_n_&nbsp;repetitions of the preceding RE, attempting to match as&nbsp;_few_repetitions as possible. This is the non-greedy version of the previous qualifier. For example, on the 6-character string&nbsp;`'aaaaaa'`,&nbsp;`a{3,5}`&nbsp;will match 5&nbsp;`'a'`&nbsp;characters, while&nbsp;`a{3,5}?`&nbsp;will only match 3 characters.

`\`
:    Either escapes special characters (permitting you to match characters like&nbsp;`'*'`,&nbsp;`'?'`, and so forth), or signals a special sequence; special sequences are discussed below.
    
If you’re not using a raw string to express the pattern, remember that Python also uses the backslash as an escape sequence in string literals; if the escape sequence isn’t recognized by Python’s parser, the backslash and subsequent character are included in the resulting string. However, if Python would recognize the resulting sequence, the backslash should be repeated twice. This is complicated and hard to understand, so it’s highly recommended that you use raw strings for all but the simplest expressions.

`[]`
:Used to indicate a set of characters. In a set:

*  Characters can be listed individually, e.g.&nbsp;`[amk]`&nbsp;will match&nbsp;`'a'`,&nbsp;`'m'`, or&nbsp;`'k'`.

*  Ranges of characters can be indicated by giving two characters and separating them by a&nbsp;`'-'`, for example&nbsp;`[a-z]`&nbsp;will match any lowercase ASCII letter,&nbsp;`[0-5][0-9]`&nbsp;will match all the two-digits numbers from&nbsp;`00`&nbsp;to&nbsp;`59`, and&nbsp;`[0-9A-Fa-f]`&nbsp;will match any hexadecimal digit. If&nbsp;`-`&nbsp;is escaped (e.g.&nbsp;`[a\-z]`) or if it’s placed as the first or last character (e.g.&nbsp;`[-a]`&nbsp;or&nbsp;`[a-]`), it will match a literal&nbsp;`'-'`.
*  Special characters lose their special meaning inside sets. For example,&nbsp;`[(+*)]`&nbsp;will match any of the literal characters&nbsp;`'('`,&nbsp;`'+'`,&nbsp;`'*'`, or&nbsp;`')'`.

*  Character classes such as&nbsp;`\w`&nbsp;or&nbsp;`\S`&nbsp;(defined below) are also accepted inside a set, although the characters they match depends on whether&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;or&nbsp;[`LOCALE`](https://docs.python.org/3/library/re.html#re.LOCALE "re.LOCALE")&nbsp;mode is in force.

*  Characters that are not within a range can be matched by&nbsp;_complementing_&nbsp;the set. If the first character of the set is&nbsp;`'^'`, all the characters that are&nbsp;_not_&nbsp;in the set will be matched. For example,&nbsp;`[^5]`&nbsp;will match any character except&nbsp;`'5'`, and&nbsp;`[^^]`&nbsp;will match any character except&nbsp;`'^'`.&nbsp;`^`&nbsp;has no special meaning if it’s not the first character in the set.
*  To match a literal&nbsp;`']'`&nbsp;inside a set, precede it with a backslash, or place it at the beginning of the set. For example, both&nbsp;`[()[\]{}]`&nbsp;and&nbsp;`[]()[{}]`&nbsp;will both match a parenthesis.

*  Support of nested sets and set operations as in&nbsp;[Unicode Technical Standard #18](https://unicode.org/reports/tr18/)&nbsp;might be added in the future. This would change the syntax, so to facilitate this change a&nbsp;[`FutureWarning`](https://docs.python.org/3/library/exceptions.html#FutureWarning "FutureWarning")&nbsp;will be raised in ambiguous cases for the time being. That includes sets starting with a literal&nbsp;`'['`&nbsp;or containing literal character sequences&nbsp;`'--'`,&nbsp;`'&&'`,&nbsp;`'~~'`, and&nbsp;`'||'`. To avoid a warning escape them with a backslash.

Changed in version 3.7:&nbsp;[`FutureWarning`](https://docs.python.org/3/library/exceptions.html#FutureWarning "FutureWarning")&nbsp;is raised if a character set contains constructs that will change semantically in the future.

`|`
:`A|B`, where&nbsp;_A_&nbsp;and&nbsp;_B_&nbsp;can be arbitrary REs, creates a regular expression that will match either&nbsp;_A_&nbsp;or&nbsp;_B_. An arbitrary number of REs can be separated by the&nbsp;`'|'`&nbsp;in this way. This can be used inside groups (see below) as well. As the target string is scanned, REs separated by&nbsp;`'|'`&nbsp;are tried from left to right. When one pattern completely matches, that branch is accepted. This means that once&nbsp;_A_&nbsp;matches,&nbsp;_B_&nbsp;will not be tested further, even if it would produce a longer overall match. In other words, the&nbsp;`'|'`&nbsp;operator is never greedy. To match a literal&nbsp;`'|'`, use&nbsp;`\|`, or enclose it inside a character class, as in&nbsp;`[|]`.

`(...)`
:    Matches whatever regular expression is inside the parentheses, and indicates the start and end of a group; the contents of a group can be retrieved after a match has been performed, and can be matched later in the string with the&nbsp;`\number`&nbsp;special sequence, described below. To match the literals&nbsp;`'('`&nbsp;or&nbsp;`')'`, use&nbsp;`\(`&nbsp;or&nbsp;`\)`, or enclose them inside a character class:&nbsp;`[(]`,&nbsp;`[)]`.

`(?...)`
:    This is an extension notation (a&nbsp;`'?'`&nbsp;following a&nbsp;`'('`&nbsp;is not meaningful otherwise). The first character after the&nbsp;`'?'`&nbsp;determines what the meaning and further syntax of the construct is. Extensions usually do not create a new group;&nbsp;`(?P<name>...)`&nbsp;is the only exception to this rule. Following are the currently supported extensions.

`(?aiLmsux)`
:    (One or more letters from the set&nbsp;`'a'`,&nbsp;`'i'`,&nbsp;`'L'`,&nbsp;`'m'`,&nbsp;`'s'`,&nbsp;`'u'`,&nbsp;`'x'`.) The group matches the empty string; the letters set the corresponding flags:&nbsp;[`re.A`](https://docs.python.org/3/library/re.html#re.A "re.A")&nbsp;(ASCII-only matching),&nbsp;[`re.I`](https://docs.python.org/3/library/re.html#re.I "re.I")&nbsp;(ignore case),&nbsp;[`re.L`](https://docs.python.org/3/library/re.html#re.L "re.L")&nbsp;(locale dependent),[`re.M`](https://docs.python.org/3/library/re.html#re.M "re.M")&nbsp;(multi-line),&nbsp;[`re.S`](https://docs.python.org/3/library/re.html#re.S "re.S")&nbsp;(dot matches all),&nbsp;`re.U`&nbsp;(Unicode matching), and&nbsp;[`re.X`](https://docs.python.org/3/library/re.html#re.X "re.X")&nbsp;(verbose), for the entire regular expression. (The flags are described in&nbsp;[Module Contents](https://docs.python.org/3/library/re.html#contents-of-module-re).) This is useful if you wish to include the flags as part of the regular expression, instead of passing a&nbsp;_flag_&nbsp;argument to the&nbsp;[`re.compile()`](https://docs.python.org/3/library/re.html#re.compile "re.compile")&nbsp;function. Flags should be used first in the expression string.

`(?:...)`
:    A non-capturing version of regular parentheses. Matches whatever regular expression is inside the parentheses, but the substring matched by the group&nbsp;_cannot_&nbsp;be retrieved after performing a match or referenced later in the pattern.

`(?aiLmsux-imsx:...)`
:    (Zero or more letters from the set&nbsp;`'a'`,&nbsp;`'i'`,&nbsp;`'L'`,&nbsp;`'m'`,&nbsp;`'s'`,&nbsp;`'u'`,&nbsp;`'x'`, optionally followed by&nbsp;`'-'`&nbsp;followed by one or more letters from the&nbsp;`'i'`,&nbsp;`'m'`,&nbsp;`'s'`,&nbsp;`'x'`.) The letters set or remove the corresponding flags:&nbsp;[`re.A`](https://docs.python.org/3/library/re.html#re.A "re.A")(ASCII-only matching),&nbsp;[`re.I`](https://docs.python.org/3/library/re.html#re.I "re.I")&nbsp;(ignore case),&nbsp;[`re.L`](https://docs.python.org/3/library/re.html#re.L "re.L")&nbsp;(locale dependent),&nbsp;[`re.M`](https://docs.python.org/3/library/re.html#re.M "re.M")&nbsp;(multi-line),&nbsp;[`re.S`](https://docs.python.org/3/library/re.html#re.S "re.S")&nbsp;(dot matches all),&nbsp;`re.U`&nbsp;(Unicode matching), and&nbsp;[`re.X`](https://docs.python.org/3/library/re.html#re.X "re.X")&nbsp;(verbose), for the part of the expression. (The flags are described in&nbsp;[Module Contents](https://docs.python.org/3/library/re.html#contents-of-module-re).)
    
The letters&nbsp;`'a'`,&nbsp;`'L'`&nbsp;and&nbsp;`'u'`&nbsp;are mutually exclusive when used as inline flags, so they can’t be combined or follow&nbsp;`'-'`. Instead, when one of them appears in an inline group, it overrides the matching mode in the enclosing group. In Unicode patterns&nbsp;`(?a:...)`&nbsp;switches to ASCII-only matching, and&nbsp;`(?u:...)`&nbsp;switches to Unicode matching (default). In byte pattern&nbsp;`(?L:...)`&nbsp;switches to locale depending matching, and&nbsp;`(?a:...)`switches to ASCII-only matching (default). This override is only in effect for the narrow inline group, and the original matching mode is restored outside of the group.

New in version 3.6.

Changed in version 3.7:&nbsp;The letters&nbsp;`'a'`,&nbsp;`'L'`&nbsp;and&nbsp;`'u'`&nbsp;also can be used in a group.

`(?P<name>...)`
:    Similar to regular parentheses, but the substring matched by the group is accessible via the symbolic group name&nbsp;_name_. Group names must be valid Python identifiers, and each group name must be defined only once within a regular expression. A symbolic group is also a numbered group, just as if the group were not named.
    
Named groups can be referenced in three contexts. If the pattern is&nbsp;`(?P<quote>['"]).*?(?P=quote)`&nbsp;(i.e. matching a string quoted with either single or double quotes):
    
| Context of reference to group “quote” | Ways to reference it |
|---------------------------------------|----------------------|
| in the same pattern itself | *  `(?P=quote)`&nbsp;(as shown)<br>*  `\1` |
| when processing match object&nbsp;_m_ | *  `m.group('quote')`<br>*  `m.end('quote')`&nbsp;(etc.) |
| in a string passed to the&nbsp;_repl_&nbsp;argument of&nbsp;`re.sub()` | *  `\g<quote>`<br>*  `\g<1>`<br>*  `\1` |

`(?P=name)`
:    A backreference to a named group; it matches whatever text was matched by the earlier group named&nbsp;_name_.

`(?#...)`
:    A comment; the contents of the parentheses are simply ignored.

`(?=...)`
:    Matches if&nbsp;`...`&nbsp;matches next, but doesn’t consume any of the string. This is called a&nbsp;_lookahead assertion_. For example,&nbsp;`Isaac (?=Asimov)`&nbsp;will match&nbsp;`'Isaac '`&nbsp;only if it’s followed by&nbsp;`'Asimov'`.

`(?!...)`
:    Matches if&nbsp;`...`&nbsp;doesn’t match next. This is a&nbsp;_negative lookahead assertion_. For example,&nbsp;`Isaac (?!Asimov)`will match&nbsp;`'Isaac '`&nbsp;only if it’s&nbsp;_not_&nbsp;followed by&nbsp;`'Asimov'`.

`(?<=...)`
:    Matches if the current position in the string is preceded by a match for&nbsp;`...`&nbsp;that ends at the current position. This is called a&nbsp;_positive lookbehind assertion_.&nbsp;`(?<=abc)def`&nbsp;will find a match in&nbsp;`'abcdef'`, since the lookbehind will back up 3 characters and check if the contained pattern matches. The contained pattern must only match strings of some fixed length, meaning that&nbsp;`abc`&nbsp;or&nbsp;`a|b`&nbsp;are allowed, but&nbsp;`a*`&nbsp;and&nbsp;`a{3,4}`&nbsp;are not. Note that patterns which start with positive lookbehind assertions will not match at the beginning of the string being searched; you will most likely want to use the&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.search "re.search")&nbsp;function rather than the&nbsp;[`match()`](https://docs.python.org/3/library/re.html#re.match "re.match")&nbsp;function:

In [1]:
import re
m = re.search('(?<=abc)def', 'abcdef')
m.group(0)

'def'

This example looks for a word following a hyphen:

In [2]:
m = re.search(r'(?<=-)\w+', 'spam-egg')
m.group(0)

'egg'

Changed in version 3.5:&nbsp;Added support for group references of fixed length.

`(?<!...)`
:    Matches if the current position in the string is not preceded by a match for&nbsp;`...`. This is called a&nbsp;_negative lookbehind assertion_. Similar to positive lookbehind assertions, the contained pattern must only match strings of some fixed length. Patterns which start with negative lookbehind assertions may match at the beginning of the string being searched.

`(?(id/name)yes-pattern|no-pattern)`
:    Will try to match with&nbsp;`yes-pattern`&nbsp;if the group with given&nbsp;_id_&nbsp;or&nbsp;_name_&nbsp;exists, and with&nbsp;`no-pattern`&nbsp;if it doesn’t.&nbsp;`no-pattern`&nbsp;is optional and can be omitted. For example,&nbsp;`(<)?(\w+@\w+(?:\.\w+)+)(?(1)>|$)`&nbsp;is a poor email matching pattern, which will match with&nbsp;`'<user@host.com>'`&nbsp;as well as&nbsp;`'user@host.com'`, but not with&nbsp;`'<user@host.com'`&nbsp;nor&nbsp;`'user@host.com>'`.

The special sequences consist of&nbsp;`'\'`&nbsp;and a character from the list below. If the ordinary character is not an ASCII digit or an ASCII letter, then the resulting RE will match the second character. For example,&nbsp;`\$`&nbsp;matches the character&nbsp;`'$'`.

`\number`
:    Matches the contents of the group of the same number. Groups are numbered starting from 1. For example,&nbsp;`(.+) \1`&nbsp;matches&nbsp;`'the the'`&nbsp;or&nbsp;`'55 55'`, but not&nbsp;`'thethe'`&nbsp;(note the space after the group). This special sequence can only be used to match one of the first 99 groups. If the first digit of&nbsp;_number_&nbsp;is 0, or&nbsp;_number_&nbsp;is 3 octal digits long, it will not be interpreted as a group match, but as the character with octal value&nbsp;_number_. Inside the&nbsp;`'['`&nbsp;and&nbsp;`']'`&nbsp;of a character class, all numeric escapes are treated as characters.

`\A`
:    Matches only at the start of the string.

`\b`
:    Matches the empty string, but only at the beginning or end of a word. A word is defined as a sequence of word characters. Note that formally,&nbsp;`\b`&nbsp;is defined as the boundary between a&nbsp;`\w`&nbsp;and a&nbsp;`\W`&nbsp;character (or vice versa), or between&nbsp;`\w`&nbsp;and the beginning/end of the string. This means that&nbsp;`r'\bfoo\b'`&nbsp;matches&nbsp;`'foo'`,&nbsp;`'foo.'`,&nbsp;`'(foo)'`,&nbsp;`'bar foo baz'`&nbsp;but not&nbsp;`'foobar'`&nbsp;or&nbsp;`'foo3'`.
    
By default Unicode alphanumerics are the ones used in Unicode patterns, but this can be changed by using the&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag. Word boundaries are determined by the current locale if the&nbsp;[`LOCALE`](https://docs.python.org/3/library/re.html#re.LOCALE "re.LOCALE")&nbsp;flag is used. Inside a character range,&nbsp;`\b`&nbsp;represents the backspace character, for compatibility with Python’s string literals.

`\B`
:    Matches the empty string, but only when it is&nbsp;_not_&nbsp;at the beginning or end of a word. This means that&nbsp;`r'py\B'`matches&nbsp;`'python'`,&nbsp;`'py3'`,&nbsp;`'py2'`, but not&nbsp;`'py'`,&nbsp;`'py.'`, or&nbsp;`'py!'`.&nbsp;`\B`&nbsp;is just the opposite of&nbsp;`\b`, so word characters in Unicode patterns are Unicode alphanumerics or the underscore, although this can be changed by using the&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag. Word boundaries are determined by the current locale if the&nbsp;[`LOCALE`](https://docs.python.org/3/library/re.html#re.LOCALE "re.LOCALE")&nbsp;flag is used.

`\d`
:    For Unicode (str) patterns:
:    Matches any Unicode decimal digit (that is, any character in Unicode character category [Nd]). This includes&nbsp;`[0-9]`, and also many other digit characters. If the&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag is used only&nbsp;`[0-9]`&nbsp;is matched.
    
:For 8-bit (bytes) patterns:
:    Matches any decimal digit; this is equivalent to&nbsp;`[0-9]`.

`\D`
:    Matches any character which is not a decimal digit. This is the opposite of&nbsp;`\d`. If the&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag is used this becomes the equivalent of&nbsp;`[^0-9]`.

`\s`
:    For Unicode (str) patterns:
:    Matches Unicode whitespace characters (which includes&nbsp;`[ \t\n\r\f\v]`, and also many other characters, for example the non-breaking spaces mandated by typography rules in many languages). If the&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag is used, only&nbsp;`[ \t\n\r\f\v]`&nbsp;is matched.
    
For 8-bit (bytes) patterns:
:    Matches characters considered whitespace in the ASCII character set; this is equivalent to&nbsp;`[\t\n\r\f\v]`.

`\S`
:    Matches any character which is not a whitespace character. This is the opposite of&nbsp;`\s`. If the&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag is used this becomes the equivalent of&nbsp;`[^ \t\n\r\f\v]`.

`\w`
:    For Unicode (str) patterns:
:    Matches Unicode word characters; this includes most characters that can be part of a word in any language, as well as numbers and the underscore. If the&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag is used, only&nbsp;`[a-zA-Z0-9_]`&nbsp;is matched.
    
For 8-bit (bytes) patterns:
:    Matches characters considered alphanumeric in the ASCII character set; this is equivalent to&nbsp;`[a-zA-Z0-9_]`. If the&nbsp;[`LOCALE`](https://docs.python.org/3/library/re.html#re.LOCALE "re.LOCALE")&nbsp;flag is used, matches characters considered alphanumeric in the current locale and the underscore.

`\W`
:    Matches any character which is not a word character. This is the opposite of&nbsp;`\w`. If the&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag is used this becomes the equivalent of&nbsp;`[^a-zA-Z0-9_]`. If the&nbsp;[`LOCALE`](https://docs.python.org/3/library/re.html#re.LOCALE "re.LOCALE")&nbsp;flag is used, matches characters which are neither alphanumeric in the current locale nor the underscore.

`\Z`
:    Matches only at the end of the string.

Most of the standard escapes supported by Python string literals are also accepted by the regular expression parser:

```
\a      \b      \f      \n  
\N      \r      \t      \u  
\U      \v      \x      \\  
```

(Note that&nbsp;`\b`&nbsp;is used to represent word boundaries, and means “backspace” only inside character classes.)

`'\u'`,&nbsp;`'\U'`, and&nbsp;`'\N'`&nbsp;escape sequences are only recognized in Unicode patterns. In bytes patterns they are errors. Unknown escapes of ASCII letters are reserved for future use and treated as errors.

Octal escapes are included in a limited form. If the first digit is a 0, or if there are three octal digits, it is considered an octal escape. Otherwise, it is a group reference. As for string literals, octal escapes are always at most three digits in length.

Changed in version 3.3:&nbsp;The&nbsp;`'\u'`&nbsp;and&nbsp;`'\U'`&nbsp;escape sequences have been added.

Changed in version 3.6:&nbsp;Unknown escapes consisting of&nbsp;`'\'`&nbsp;and an ASCII letter now are errors.

Changed in version 3.8:&nbsp;The&nbsp;`'\N{name}'`&nbsp;escape sequence has been added. As in string literals, it expands to the named Unicode character (e.g.&nbsp;`'\N{EM DASH}'`).

## Module Contents[](https://docs.python.org/3/library/re.html#module-contents "Permalink to this headline")

The module defines several functions, constants, and an exception. Some of the functions are simplified versions of the full featured methods for compiled regular expressions. Most non-trivial applications always use the compiled form.

Changed in version 3.6:&nbsp;Flag constants are now instances of&nbsp;`RegexFlag`, which is a subclass of&nbsp;[`enum.IntFlag`](https://docs.python.org/3/library/enum.html#enum.IntFlag "enum.IntFlag").

`re.compile`(_pattern_,&nbsp;_flags=0_)[](https://docs.python.org/3/library/re.html#re.compile "Permalink to this definition")
:    Compile a regular expression pattern into a&nbsp;[regular expression object](https://docs.python.org/3/library/re.html#re-objects), which can be used for matching using its&nbsp;[`match()`](https://docs.python.org/3/library/re.html#re.Pattern.match "re.Pattern.match"),&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.Pattern.search "re.Pattern.search")&nbsp;and other methods, described below.
    
The expression’s behaviour can be modified by specifying a&nbsp;_flags_&nbsp;value. Values can be any of the following variables, combined using bitwise OR (the&nbsp;`|`&nbsp;operator).
    
The sequence

```
prog = re.compile(pattern)
result = prog.match(string)

is equivalent to

```
result = re.match(pattern, string)
```

but using&nbsp;[`re.compile()`](https://docs.python.org/3/library/re.html#re.compile "re.compile")&nbsp;and saving the resulting regular expression object for reuse is more efficient when the expression will be used several times in a single program.
    
Note
    
&nbsp;
    
The compiled versions of the most recent patterns passed to&nbsp;[`re.compile()`](https://docs.python.org/3/library/re.html#re.compile "re.compile")&nbsp;and the module-level matching functions are cached, so programs that use only a few regular expressions at a time needn’t worry about compiling regular expressions.

`re.A`[](https://docs.python.org/3/library/re.html#re.A "Permalink to this definition")

`re.ASCII`[](https://docs.python.org/3/library/re.html#re.ASCII "Permalink to this definition")
:    Make&nbsp;`\w`,&nbsp;`\W`,&nbsp;`\b`,&nbsp;`\B`,&nbsp;`\d`,&nbsp;`\D`,&nbsp;`\s`&nbsp;and&nbsp;`\S`&nbsp;perform ASCII-only matching instead of full Unicode matching. This is only meaningful for Unicode patterns, and is ignored for byte patterns. Corresponds to the inline flag&nbsp;`(?a)`.
    
Note that for backward compatibility, the&nbsp;`re.U`&nbsp;flag still exists (as well as its synonym&nbsp;`re.UNICODE`&nbsp;and its embedded counterpart&nbsp;`(?u)`), but these are redundant in Python 3 since matches are Unicode by default for strings (and Unicode matching isn’t allowed for bytes).

`re.DEBUG`[](https://docs.python.org/3/library/re.html#re.DEBUG "Permalink to this definition")
:    Display debug information about compiled expression. No corresponding inline flag.

`re.I`[](https://docs.python.org/3/library/re.html#re.I "Permalink to this definition")

`re.IGNORECASE`[](https://docs.python.org/3/library/re.html#re.IGNORECASE "Permalink to this definition")
:    Perform case-insensitive matching; expressions like&nbsp;`[A-Z]`&nbsp;will also match lowercase letters. Full Unicode matching (such as&nbsp;`Ü`&nbsp;matching&nbsp;`ü`) also works unless the&nbsp;[`re.ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag is used to disable non-ASCII matches. The current locale does not change the effect of this flag unless the&nbsp;[`re.LOCALE`](https://docs.python.org/3/library/re.html#re.LOCALE "re.LOCALE")&nbsp;flag is also used. Corresponds to the inline flag&nbsp;`(?i)`.
    
Note that when the Unicode patterns&nbsp;`[a-z]`&nbsp;or&nbsp;`[A-Z]`&nbsp;are used in combination with the&nbsp;[`IGNORECASE`](https://docs.python.org/3/library/re.html#re.IGNORECASE "re.IGNORECASE")&nbsp;flag, they will match the 52 ASCII letters and 4 additional non-ASCII letters: ‘İ’ (U+0130, Latin capital letter I with dot above), ‘ı’ (U+0131, Latin small letter dotless i), ‘ſ’ (U+017F, Latin small letter long s) and ‘K’ (U+212A, Kelvin sign). If the&nbsp;[`ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII")&nbsp;flag is used, only letters ‘a’ to ‘z’ and ‘A’ to ‘Z’ are matched.

`re.L`[](https://docs.python.org/3/library/re.html#re.L "Permalink to this definition")

`re.LOCALE`[](https://docs.python.org/3/library/re.html#re.LOCALE "Permalink to this definition")
:    Make&nbsp;`\w`,&nbsp;`\W`,&nbsp;`\b`,&nbsp;`\B`&nbsp;and case-insensitive matching dependent on the current locale. This flag can be used only with bytes patterns. The use of this flag is discouraged as the locale mechanism is very unreliable, it only handles one “culture” at a time, and it only works with 8-bit locales. Unicode matching is already enabled by default in Python 3 for Unicode (str) patterns, and it is able to handle different locales/languages. Corresponds to the inline flag&nbsp;`(?L)`.
    
Changed in version 3.6:&nbsp;[`re.LOCALE`](https://docs.python.org/3/library/re.html#re.LOCALE "re.LOCALE")&nbsp;can be used only with bytes patterns and is not compatible with&nbsp;[`re.ASCII`](https://docs.python.org/3/library/re.html#re.ASCII "re.ASCII").
    
Changed in version 3.7:&nbsp;Compiled regular expression objects with the&nbsp;[`re.LOCALE`](https://docs.python.org/3/library/re.html#re.LOCALE "re.LOCALE")&nbsp;flag no longer depend on the locale at compile time. Only the locale at matching time affects the result of matching.

`re.M`[](https://docs.python.org/3/library/re.html#re.M "Permalink to this definition")

`re.MULTILINE`[](https://docs.python.org/3/library/re.html#re.MULTILINE "Permalink to this definition")
:    When specified, the pattern character&nbsp;`'^'`&nbsp;matches at the beginning of the string and at the beginning of each line (immediately following each newline); and the pattern character&nbsp;`'$'`&nbsp;matches at the end of the string and at the end of each line (immediately preceding each newline). By default,&nbsp;`'^'`&nbsp;matches only at the beginning of the string, and&nbsp;`'$'`&nbsp;only at the end of the string and immediately before the newline (if any) at the end of the string. Corresponds to the inline flag&nbsp;`(?m)`.

`re.S`[](https://docs.python.org/3/library/re.html#re.S "Permalink to this definition")

`re.DOTALL`[](https://docs.python.org/3/library/re.html#re.DOTALL "Permalink to this definition")
:    Make the&nbsp;`'.'`&nbsp;special character match any character at all, including a newline; without this flag,&nbsp;`'.'`&nbsp;will match anything&nbsp;_except_&nbsp;a newline. Corresponds to the inline flag&nbsp;`(?s)`.

`re.X`[](https://docs.python.org/3/library/re.html#re.X "Permalink to this definition")

`re.VERBOSE`[](https://docs.python.org/3/library/re.html#re.VERBOSE "Permalink to this definition")
:    This flag allows you to write regular expressions that look nicer and are more readable by allowing you to visually separate logical sections of the pattern and add comments. Whitespace within the pattern is ignored, except when in a character class, or when preceded by an unescaped backslash, or within tokens like&nbsp;`*?`,&nbsp;`(?:`&nbsp;or&nbsp;`(?P<...>`. When a line contains a&nbsp;`#`&nbsp;that is not in a character class and is not preceded by an unescaped backslash, all characters from the leftmost such&nbsp;`#`&nbsp;through the end of the line are ignored.
    
This means that the two following regular expression objects that match a decimal number are functionally equal:

In [3]:
a = re.compile(r"""\d +  # the integral part
                   \.    # the decimal point
                   \d *  # some fractional digits""", re.X)
b = re.compile(r"\d+\.\d*")

Corresponds to the inline flag&nbsp;`(?x)`.

`re.search`(_pattern_,&nbsp;_string_,&nbsp;_flags=0_)[](https://docs.python.org/3/library/re.html#re.search "Permalink to this definition")
:    Scan through&nbsp;_string_&nbsp;looking for the first location where the regular expression&nbsp;_pattern_&nbsp;produces a match, and return a corresponding&nbsp;[match object](https://docs.python.org/3/library/re.html#match-objects). Return&nbsp;`None`&nbsp;if no position in the string matches the pattern; note that this is different from finding a zero-length match at some point in the string.

`re.match`(_pattern_,&nbsp;_string_,&nbsp;_flags=0_)[](https://docs.python.org/3/library/re.html#re.match "Permalink to this definition")
:    If zero or more characters at the beginning of&nbsp;_string_&nbsp;match the regular expression&nbsp;_pattern_, return a corresponding&nbsp;[match object](https://docs.python.org/3/library/re.html#match-objects). Return&nbsp;`None`&nbsp;if the string does not match the pattern; note that this is different from a zero-length match.
    
Note that even in&nbsp;[`MULTILINE`](https://docs.python.org/3/library/re.html#re.MULTILINE "re.MULTILINE")&nbsp;mode,&nbsp;[`re.match()`](https://docs.python.org/3/library/re.html#re.match "re.match")&nbsp;will only match at the beginning of the string and not at the beginning of each line.
    
If you want to locate a match anywhere in&nbsp;_string_, use&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.search "re.search")&nbsp;instead (see also&nbsp;[search() vs. match()](https://docs.python.org/3/library/re.html#search-vs-match)).

`re.fullmatch`(_pattern_,&nbsp;_string_,&nbsp;_flags=0_)[](https://docs.python.org/3/library/re.html#re.fullmatch "Permalink to this definition")
:    If the whole&nbsp;_string_&nbsp;matches the regular expression&nbsp;_pattern_, return a corresponding&nbsp;[match object](https://docs.python.org/3/library/re.html#match-objects). Return&nbsp;`None`if the string does not match the pattern; note that this is different from a zero-length match.
    
New in version 3.4.

`re.split`(_pattern_,&nbsp;_string_,&nbsp;_maxsplit=0_,&nbsp;_flags=0_)[](https://docs.python.org/3/library/re.html#re.split "Permalink to this definition")
:    Split&nbsp;_string_&nbsp;by the occurrences of&nbsp;_pattern_. If capturing parentheses are used in&nbsp;_pattern_, then the text of all groups in the pattern are also returned as part of the resulting list. If&nbsp;_maxsplit_&nbsp;is nonzero, at most&nbsp;_maxsplit_splits occur, and the remainder of the string is returned as the final element of the list.

In [4]:
re.split(r'\W+', 'Words, words, words.')

['Words', 'words', 'words', '']

In [5]:
re.split(r'(\W+)', 'Words, words, words.')

['Words', ', ', 'words', ', ', 'words', '.', '']

In [6]:
re.split(r'\W+', 'Words, words, words.', 1)

['Words', 'words, words.']

In [7]:
re.split('[a-f]+', '0a3B9', flags=re.IGNORECASE)

['0', '3', '9']

If there are capturing groups in the separator and it matches at the start of the string, the result will start with an empty string. The same holds for the end of the string:


In [8]:
re.split(r'(\W+)', '...words, words...')

['', '...', 'words', ', ', 'words', '...', '']

That way, separator components are always found at the same relative indices within the result list.
    
Empty matches for the pattern split the string only when not adjacent to a previous empty match.

In [9]:
re.split(r'\b', 'Words, words, words.')

ValueError: split() requires a non-empty pattern match.

In [10]:
re.split(r'\W*', '...words...')

C:\ProgramData\Anaconda3\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


['', 'words', '']

In [11]:
re.split(r'(\W*)', '...words...')

C:\ProgramData\Anaconda3\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


['', '...', 'words', '...', '']

Changed in version 3.1:&nbsp;Added the optional flags argument.
    
Changed in version 3.7:&nbsp;Added support of splitting on a pattern that could match an empty string.

`re.findall`(_pattern_,&nbsp;_string_,&nbsp;_flags=0_)[](https://docs.python.org/3/library/re.html#re.findall "Permalink to this definition")
:    Return all non-overlapping matches of&nbsp;_pattern_&nbsp;in&nbsp;_string_, as a list of strings. The&nbsp;_string_&nbsp;is scanned left-to-right, and matches are returned in the order found. If one or more groups are present in the pattern, return a list of groups; this will be a list of tuples if the pattern has more than one group. Empty matches are included in the result.
    
Changed in version 3.7:&nbsp;Non-empty matches can now start just after a previous empty match.

`re.finditer`(_pattern_,&nbsp;_string_,&nbsp;_flags=0_)[](https://docs.python.org/3/library/re.html#re.finditer "Permalink to this definition")
:    Return an&nbsp;[iterator](https://docs.python.org/3/glossary.html#term-iterator)&nbsp;yielding&nbsp;[match objects](https://docs.python.org/3/library/re.html#match-objects)&nbsp;over all non-overlapping matches for the RE&nbsp;_pattern_&nbsp;in&nbsp;_string_. The&nbsp;_string_&nbsp;is scanned left-to-right, and matches are returned in the order found. Empty matches are included in the result.
    
Changed in version 3.7:&nbsp;Non-empty matches can now start just after a previous empty match.

`re.sub`(_pattern_,&nbsp;_repl_,&nbsp;_string_,&nbsp;_count=0_,&nbsp;_flags=0_)[](https://docs.python.org/3/library/re.html#re.sub "Permalink to this definition")
:    Return the string obtained by replacing the leftmost non-overlapping occurrences of&nbsp;_pattern_&nbsp;in&nbsp;_string_&nbsp;by the replacement&nbsp;_repl_. If the pattern isn’t found,&nbsp;_string_&nbsp;is returned unchanged.&nbsp;_repl_&nbsp;can be a string or a function; if it is a string, any backslash escapes in it are processed. That is,&nbsp;`\n`&nbsp;is converted to a single newline character,&nbsp;`\r`&nbsp;is converted to a carriage return, and so forth. Unknown escapes of ASCII letters are reserved for future use and treated as errors. Other unknown escapes such as&nbsp;`\&`&nbsp;are left alone. Backreferences, such as&nbsp;`\6`, are replaced with the substring matched by group 6 in the pattern. For example:


In [15]:
re.sub(r'def\s+([a-zA-Z_][a-zA-Z_0-9]*)\s*\(\s*\):',
       r'static PyObject*\npy_\1(void)\n{',
       'def myfunc():')

'static PyObject*\npy_myfunc(void)\n{'

If&nbsp;_repl_&nbsp;is a function, it is called for every non-overlapping occurrence of&nbsp;_pattern_. The function takes a single&nbsp;[match object](https://docs.python.org/3/library/re.html#match-objects)&nbsp;argument, and returns the replacement string. For example:

In [13]:
def dashrepl(matchobj):
    if matchobj.group(0) == '-': return ' '
    else: return '-'
re.sub('-{1,2}', dashrepl, 'pro----gram-files')

'pro--gram files'

In [14]:
re.sub(r'\sAND\s', ' & ', 'Baked Beans And Spam', flags=re.IGNORECASE)

'Baked Beans & Spam'

The pattern may be a string or a&nbsp;[pattern object](https://docs.python.org/3/library/re.html#re-objects).
    
The optional argument&nbsp;_count_&nbsp;is the maximum number of pattern occurrences to be replaced;&nbsp;_count_&nbsp;must be a non-negative integer. If omitted or zero, all occurrences will be replaced. Empty matches for the pattern are replaced only when not adjacent to a previous empty match, so&nbsp;`sub('x*', '-', 'abxd')`&nbsp;returns&nbsp;`'-a-b--d-'`.

In string-type&nbsp;_repl_&nbsp;arguments, in addition to the character escapes and backreferences described above,`\g<name>`&nbsp;will use the substring matched by the group named&nbsp;`name`, as defined by the&nbsp;`(?P<name>...)`&nbsp;syntax.&nbsp;`\g<number>`&nbsp;uses the corresponding group number;&nbsp;`\g<2>`&nbsp;is therefore equivalent to&nbsp;`\2`, but isn’t ambiguous in a replacement such as&nbsp;`\g<2>0`.&nbsp;`\20`&nbsp;would be interpreted as a reference to group 20, not a reference to group 2 followed by the literal character&nbsp;`'0'`. The backreference&nbsp;`\g<0>`&nbsp;substitutes in the entire substring matched by the RE.

Changed in version 3.1:&nbsp;Added the optional flags argument.

Changed in version 3.5:&nbsp;Unmatched groups are replaced with an empty string.

Changed in version 3.6:&nbsp;Unknown escapes in&nbsp;_pattern_&nbsp;consisting of&nbsp;`'\'`&nbsp;and an ASCII letter now are errors.

Changed in version 3.7:&nbsp;Unknown escapes in&nbsp;_repl_&nbsp;consisting of&nbsp;`'\'`&nbsp;and an ASCII letter now are errors.

Changed in version 3.7:&nbsp;Empty matches for the pattern are replaced when adjacent to a previous non-empty match.

`re.subn`(_pattern_,&nbsp;_repl_,&nbsp;_string_,&nbsp;_count=0_,&nbsp;_flags=0_)[](https://docs.python.org/3/library/re.html#re.subn "Permalink to this definition")
:Perform the same operation as&nbsp;[`sub()`](https://docs.python.org/3/library/re.html#re.sub "re.sub"), but return a tuple&nbsp;`(new_string, number_of_subs_made)`.

Changed in version 3.1:&nbsp;Added the optional flags argument.

Changed in version 3.5:&nbsp;Unmatched groups are replaced with an empty string.

`re.escape`(_pattern_)[](https://docs.python.org/3/library/re.html#re.escape "Permalink to this definition")
:Escape special characters in&nbsp;_pattern_. This is useful if you want to match an arbitrary literal string that may have regular expression metacharacters in it. For example:
    


In [16]:
print(re.escape('http://www.python.org'))

http\:\/\/www\.python\.org


In [19]:
import string
legal_chars = string.ascii_lowercase + string.digits + "!#$%&'*+-.^_`|~:"
print('[%s]+' % re.escape(legal_chars))

[abcdefghijklmnopqrstuvwxyz0123456789\!\#\$\%\&\'\*\+\-\.\^_\`\|\~\:]+


In [20]:
operators = ['+', '-', '*', '/', '**']
print('|'.join(map(re.escape, sorted(operators, reverse=True))))

\/|\-|\+|\*\*|\*


This function must not be used for the replacement string in&nbsp;[`sub()`](https://docs.python.org/3/library/re.html#re.sub "re.sub")&nbsp;and&nbsp;[`subn()`](https://docs.python.org/3/library/re.html#re.subn "re.subn"), only backslashes should be escaped. For example:


In [21]:
digits_re = r'\d+'
sample = '/usr/sbin/sendmail - 0 errors, 12 warnings'
print(re.sub(digits_re, digits_re.replace('\\', r'\\'), sample))

/usr/sbin/sendmail - \d+ errors, \d+ warnings


Changed in version 3.3:&nbsp;The&nbsp;`'_'`&nbsp;character is no longer escaped.
    
Changed in version 3.7:&nbsp;Only characters that can have special meaning in a regular expression are escaped. As a result,&nbsp;`'!'`,&nbsp;`'"'`,&nbsp;`'%'`,&nbsp;`"'"`,&nbsp;`','`,&nbsp;`'/'`,&nbsp;`':'`,&nbsp;`';'`,&nbsp;`'<'`,&nbsp;`'='`,&nbsp;`'>'`,&nbsp;`'@'`, and&nbsp;`"`"`&nbsp;are no longer escaped.

`re.purge`()[](https://docs.python.org/3/library/re.html#re.purge "Permalink to this definition")
:    Clear the regular expression cache.

_exception&nbsp;_`re.error`(_msg_,&nbsp;_pattern=None_,&nbsp;_pos=None_)[](https://docs.python.org/3/library/re.html#re.error "Permalink to this definition")
:    Exception raised when a string passed to one of the functions here is not a valid regular expression (for example, it might contain unmatched parentheses) or when some other error occurs during compilation or matching. It is never an error if a string contains no match for a pattern. The error instance has the following additional attributes:
    
`msg`[](https://docs.python.org/3/library/re.html#re.error.msg "Permalink to this definition")
:    The unformatted error message.

`pattern`[](https://docs.python.org/3/library/re.html#re.error.pattern "Permalink to this definition")
:    The regular expression pattern.

`pos`[](https://docs.python.org/3/library/re.html#re.error.pos "Permalink to this definition")
:    The index in&nbsp;_pattern_&nbsp;where compilation failed (may be&nbsp;`None`).

`lineno`[](https://docs.python.org/3/library/re.html#re.error.lineno "Permalink to this definition")
:    The line corresponding to&nbsp;_pos_&nbsp;(may be&nbsp;`None`).

`colno`[](https://docs.python.org/3/library/re.html#re.error.colno "Permalink to this definition")
:    The column corresponding to&nbsp;_pos_&nbsp;(may be&nbsp;`None`).

Changed in version 3.5:&nbsp;Added additional attributes.

## Regular Expression Objects[](https://docs.python.org/3/library/re.html#regular-expression-objects "Permalink to this headline")

Compiled regular expression objects support the following methods and attributes:

`Pattern.search`(_string_[,&nbsp;_pos_[,&nbsp;_endpos_]])[](https://docs.python.org/3/library/re.html#re.Pattern.search "Permalink to this definition")
:    Scan through&nbsp;_string_&nbsp;looking for the first location where this regular expression produces a match, and return a corresponding&nbsp;[match object](https://docs.python.org/3/library/re.html#match-objects). Return&nbsp;`None`&nbsp;if no position in the string matches the pattern; note that this is different from finding a zero-length match at some point in the string.
    
The optional second parameter&nbsp;_pos_&nbsp;gives an index in the string where the search is to start; it defaults to&nbsp;`0`. This is not completely equivalent to slicing the string; the&nbsp;`'^'`&nbsp;pattern character matches at the real beginning of the string and at positions just after a newline, but not necessarily at the index where the search is to start.
    
The optional parameter&nbsp;_endpos_&nbsp;limits how far the string will be searched; it will be as if the string is&nbsp;_endpos_characters long, so only the characters from&nbsp;_pos_&nbsp;to&nbsp;`endpos - 1`&nbsp;will be searched for a match. If&nbsp;_endpos_&nbsp;is less than&nbsp;_pos_, no match will be found; otherwise, if&nbsp;_rx_&nbsp;is a compiled regular expression object,&nbsp;`rx.search(string, 0, 50)`&nbsp;is equivalent to&nbsp;`rx.search(string[:50], 0)`.


In [22]:
pattern = re.compile("d")
pattern.search("dog")     # Match at index 0

<_sre.SRE_Match object; span=(0, 1), match='d'>

In [23]:
pattern.search("dog", 1)  # No match; search doesn't include the "d"

`Pattern.match`(_string_[,&nbsp;_pos_[,&nbsp;_endpos_]])[](https://docs.python.org/3/library/re.html#re.Pattern.match "Permalink to this definition")
:    If zero or more characters at the&nbsp;_beginning_&nbsp;of&nbsp;_string_&nbsp;match this regular expression, return a corresponding&nbsp;[match object](https://docs.python.org/3/library/re.html#match-objects). Return&nbsp;`None`&nbsp;if the string does not match the pattern; note that this is different from a zero-length match.
    
The optional&nbsp;_pos_&nbsp;and&nbsp;_endpos_&nbsp;parameters have the same meaning as for the&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.Pattern.search "re.Pattern.search")&nbsp;method.


In [24]:
pattern = re.compile("o")
pattern.match("dog")      # No match as "o" is not at the start of "dog".
pattern.match("dog", 1)   # Match as "o" is the 2nd character of "dog".

<_sre.SRE_Match object; span=(1, 2), match='o'>

If you want to locate a match anywhere in&nbsp;_string_, use&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.Pattern.search "re.Pattern.search")&nbsp;instead (see also&nbsp;[search() vs. match()](https://docs.python.org/3/library/re.html#search-vs-match)).

`Pattern.fullmatch`(_string_[,&nbsp;_pos_[,&nbsp;_endpos_]])[](https://docs.python.org/3/library/re.html#re.Pattern.fullmatch "Permalink to this definition")
:    If the whole&nbsp;_string_&nbsp;matches this regular expression, return a corresponding&nbsp;[match object](https://docs.python.org/3/library/re.html#match-objects). Return&nbsp;`None`&nbsp;if the string does not match the pattern; note that this is different from a zero-length match.
    
The optional&nbsp;_pos_&nbsp;and&nbsp;_endpos_&nbsp;parameters have the same meaning as for the&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.Pattern.search "re.Pattern.search")&nbsp;method.


In [25]:
pattern = re.compile("o[gh]")
pattern.fullmatch("dog")      # No match as "o" is not at the start of "dog".
pattern.fullmatch("ogre")     # No match as not the full string matches.
pattern.fullmatch("doggie", 1, 3)   # Matches within given limits.

<_sre.SRE_Match object; span=(1, 3), match='og'>

New in version 3.4.

`Pattern.split`(_string_,&nbsp;_maxsplit=0_)[](https://docs.python.org/3/library/re.html#re.Pattern.split "Permalink to this definition")
:    Identical to the&nbsp;[`split()`](https://docs.python.org/3/library/re.html#re.split "re.split")&nbsp;function, using the compiled pattern.

`Pattern.findall`(_string_[,&nbsp;_pos_[,&nbsp;_endpos_]])[](https://docs.python.org/3/library/re.html#re.Pattern.findall "Permalink to this definition")
:    Similar to the&nbsp;[`findall()`](https://docs.python.org/3/library/re.html#re.findall "re.findall")&nbsp;function, using the compiled pattern, but also accepts optional&nbsp;_pos_&nbsp;and&nbsp;_endpos_parameters that limit the search region like for&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.search "re.search").

`Pattern.finditer`(_string_[,&nbsp;_pos_[,&nbsp;_endpos_]])[](https://docs.python.org/3/library/re.html#re.Pattern.finditer "Permalink to this definition")
:    Similar to the&nbsp;[`finditer()`](https://docs.python.org/3/library/re.html#re.finditer "re.finditer")&nbsp;function, using the compiled pattern, but also accepts optional&nbsp;_pos_&nbsp;and&nbsp;_endpos_parameters that limit the search region like for&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.search "re.search").

`Pattern.sub`(_repl_,&nbsp;_string_,&nbsp;_count=0_)[](https://docs.python.org/3/library/re.html#re.Pattern.sub "Permalink to this definition")
:    Identical to the&nbsp;[`sub()`](https://docs.python.org/3/library/re.html#re.sub "re.sub")&nbsp;function, using the compiled pattern.

`Pattern.subn`(_repl_,&nbsp;_string_,&nbsp;_count=0_)[](https://docs.python.org/3/library/re.html#re.Pattern.subn "Permalink to this definition")
:    Identical to the&nbsp;[`subn()`](https://docs.python.org/3/library/re.html#re.subn "re.subn")&nbsp;function, using the compiled pattern.

`Pattern.flags`[](https://docs.python.org/3/library/re.html#re.Pattern.flags "Permalink to this definition")
:    The regex matching flags. This is a combination of the flags given to&nbsp;[`compile()`](https://docs.python.org/3/library/re.html#re.compile "re.compile"), any&nbsp;`(?...)`&nbsp;inline flags in the pattern, and implicit flags such as&nbsp;`UNICODE`&nbsp;if the pattern is a Unicode string.

`Pattern.groups`[](https://docs.python.org/3/library/re.html#re.Pattern.groups "Permalink to this definition")
:    The number of capturing groups in the pattern.

`Pattern.groupindex`[](https://docs.python.org/3/library/re.html#re.Pattern.groupindex "Permalink to this definition")
:    A dictionary mapping any symbolic group names defined by&nbsp;`(?P<id>)`&nbsp;to group numbers. The dictionary is empty if no symbolic groups were used in the pattern.

`Pattern.pattern`[](https://docs.python.org/3/library/re.html#re.Pattern.pattern "Permalink to this definition")
:    The pattern string from which the pattern object was compiled.

Changed in version 3.7:&nbsp;Added support of&nbsp;[`copy.copy()`](https://docs.python.org/3/library/copy.html#copy.copy "copy.copy")&nbsp;and&nbsp;[`copy.deepcopy()`](https://docs.python.org/3/library/copy.html#copy.deepcopy "copy.deepcopy"). Compiled regular expression objects are considered atomic.

## Match Objects[](https://docs.python.org/3/library/re.html#match-objects "Permalink to this headline")

Match objects always have a boolean value of&nbsp;`True`. Since&nbsp;[`match()`](https://docs.python.org/3/library/re.html#re.Pattern.match "re.Pattern.match")&nbsp;and&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.Pattern.search "re.Pattern.search")&nbsp;return&nbsp;`None`&nbsp;when there is no match, you can test whether there was a match with a simple&nbsp;`if`&nbsp;statement:

In [ ]:
match = re.search(pattern, string)
if match:
    process(match)

Match objects support the following methods and attributes:

`Match.expand`(_template_)[](https://docs.python.org/3/library/re.html#re.Match.expand "Permalink to this definition")
:    Return the string obtained by doing backslash substitution on the template string&nbsp;_template_, as done by the&nbsp;[`sub()`](https://docs.python.org/3/library/re.html#re.Pattern.sub "re.Pattern.sub")&nbsp;method. Escapes such as&nbsp;`\n`&nbsp;are converted to the appropriate characters, and numeric backreferences (`\1`,&nbsp;`\2`) and named backreferences (`\g<1>`,&nbsp;`\g<name>`) are replaced by the contents of the corresponding group.
    
Changed in version 3.5:&nbsp;Unmatched groups are replaced with an empty string.

`Match.group`([_group1_,&nbsp;_..._])[](https://docs.python.org/3/library/re.html#re.Match.group "Permalink to this definition")
:    Returns one or more subgroups of the match. If there is a single argument, the result is a single string; if there are multiple arguments, the result is a tuple with one item per argument. Without arguments,&nbsp;_group1_defaults to zero (the whole match is returned). If a&nbsp;_groupN_&nbsp;argument is zero, the corresponding return value is the entire matching string; if it is in the inclusive range [1..99], it is the string matching the corresponding parenthesized group. If a group number is negative or larger than the number of groups defined in the pattern, an&nbsp;[`IndexError`](https://docs.python.org/3/library/exceptions.html#IndexError "IndexError")&nbsp;exception is raised. If a group is contained in a part of the pattern that did not match, the corresponding result is&nbsp;`None`. If a group is contained in a part of the pattern that matched multiple times, the last match is returned.


In [26]:
m = re.match(r"(\w+) (\w+)", "Isaac Newton, physicist")
m.group(0)       # The entire match

'Isaac Newton'

In [27]:
m.group(1)       # The first parenthesized subgroup.

'Isaac'

In [28]:
m.group(2)       # The second parenthesized subgroup.

'Newton'

In [29]:
m.group(1, 2)    # Multiple arguments give us a tuple.

('Isaac', 'Newton')

If the regular expression uses the&nbsp;`(?P<name>...)`&nbsp;syntax, the&nbsp;_groupN_&nbsp;arguments may also be strings identifying groups by their group name. If a string argument is not used as a group name in the pattern, an&nbsp;[`IndexError`](https://docs.python.org/3/library/exceptions.html#IndexError "IndexError")&nbsp;exception is raised.
    
A moderately complicated example:


In [2]:
m = re.match(r"(?P<first_name>\w+) (?P<last_name>\w+)", "Malcolm Reynolds")
m.group('first_name')

'Malcolm'

In [31]:
m.group('last_name')

'Reynolds'

Named groups can also be referred to by their index:


In [3]:
m.group(1)

'Malcolm'

In [4]:
m.group(2)

'Reynolds'

If a group matches multiple times, only the last match is accessible:

In [5]:
m = re.match(r"(..)+", "a1b2c3")  # Matches 3 times.
m.group(1)                        # Returns only the last match.

'c3'

`Match.__getitem__`(_g_)[](https://docs.python.org/3/library/re.html#re.Match.__getitem__ "Permalink to this definition")
:    This is identical to&nbsp;`m.group(g)`. This allows easier access to an individual group from a match:

In [8]:
m = re.match(r"(\w+) (\w+)", "Isaac Newton, physicist")
m.group(0)    # The entire match

'Isaac Newton'

In [9]:
m.group(1)       # The first parenthesized subgroup.

'Isaac'

In [10]:
m.group(2)       # The second parenthesized subgroup.

'Newton'

New in version 3.6.

`Match.groups`(_default=None_)[](https://docs.python.org/3/library/re.html#re.Match.groups "Permalink to this definition")
:    Return a tuple containing all the subgroups of the match, from 1 up to however many groups are in the pattern. The&nbsp;_default_&nbsp;argument is used for groups that did not participate in the match; it defaults to&nbsp;`None`.
    
For example:

In [11]:
m = re.match(r"(\d+)\.(\d+)", "24.1632")
m.groups()

('24', '1632')

If we make the decimal place and everything after it optional, not all groups might participate in the match. These groups will default to&nbsp;`None`&nbsp;unless the&nbsp;_default_&nbsp;argument is given:

In [12]:
m = re.match(r"(\d+)\.?(\d+)?", "24")
m.groups()      # Second group defaults to None.

('24', None)

In [13]:
m.groups('0')   # Now, the second group defaults to '0'.

('24', '0')

`Match.groupdict`(_default=None_)[](https://docs.python.org/3/library/re.html#re.Match.groupdict "Permalink to this definition")
:    Return a dictionary containing all the&nbsp;_named_&nbsp;subgroups of the match, keyed by the subgroup name. The&nbsp;_default_&nbsp;argument is used for groups that did not participate in the match; it defaults to&nbsp;`None`. For example:

In [14]:
m = re.match(r"(?P<first_name>\w+) (?P<last_name>\w+)", "Malcolm Reynolds")
m.groupdict()

{'first_name': 'Malcolm', 'last_name': 'Reynolds'}

`Match.start`([_group_])[](https://docs.python.org/3/library/re.html#re.Match.start "Permalink to this definition")

`Match.end`([_group_])[](https://docs.python.org/3/library/re.html#re.Match.end "Permalink to this definition")
:    Return the indices of the start and end of the substring matched by&nbsp;_group_;&nbsp;_group_&nbsp;defaults to zero (meaning the whole matched substring). Return&nbsp;`-1`&nbsp;if&nbsp;_group_&nbsp;exists but did not contribute to the match. For a match object&nbsp;_m_, and a group&nbsp;_g_&nbsp;that did contribute to the match, the substring matched by group&nbsp;_g_&nbsp;(equivalent to&nbsp;`m.group(g)`) is

In [15]:
m.string[m.start(g):m.end(g)]

NameError: name 'g' is not defined

Note that&nbsp;`m.start(group)`&nbsp;will equal&nbsp;`m.end(group)`&nbsp;if&nbsp;_group_&nbsp;matched a null string. For example, after&nbsp;`m =re.search('b(c?)', 'cba')`,&nbsp;`m.start(0)`&nbsp;is 1,&nbsp;`m.end(0)`&nbsp;is 2,&nbsp;`m.start(1)`&nbsp;and&nbsp;`m.end(1)`&nbsp;are both 2, and&nbsp;`m.start(2)`&nbsp;raises an&nbsp;[`IndexError`](https://docs.python.org/3/library/exceptions.html#IndexError "IndexError")&nbsp;exception.
    
An example that will remove&nbsp;_remove_this_&nbsp;from email addresses:

In [16]:
email = "tony@tiremove_thisger.net"
m = re.search("remove_this", email)
email[:m.start()] + email[m.end():]

'tony@tiger.net'

`Match.span`([_group_])[](https://docs.python.org/3/library/re.html#re.Match.span "Permalink to this definition")
:    For a match&nbsp;_m_, return the 2-tuple&nbsp;`(m.start(group), m.end(group))`. Note that if&nbsp;_group_&nbsp;did not contribute to the match, this is&nbsp;`(-1, -1)`.&nbsp;_group_&nbsp;defaults to zero, the entire match.

`Match.pos`[](https://docs.python.org/3/library/re.html#re.Match.pos "Permalink to this definition")
:    The value of&nbsp;_pos_&nbsp;which was passed to the&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.Pattern.search "re.Pattern.search")&nbsp;or&nbsp;[`match()`](https://docs.python.org/3/library/re.html#re.Pattern.match "re.Pattern.match")&nbsp;method of a&nbsp;[regex object](https://docs.python.org/3/library/re.html#re-objects). This is the index into the string at which the RE engine started looking for a match.

`Match.endpos`[](https://docs.python.org/3/library/re.html#re.Match.endpos "Permalink to this definition")
:    The value of&nbsp;_endpos_&nbsp;which was passed to the&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.Pattern.search "re.Pattern.search")&nbsp;or&nbsp;[`match()`](https://docs.python.org/3/library/re.html#re.Pattern.match "re.Pattern.match")&nbsp;method of a&nbsp;[regex object](https://docs.python.org/3/library/re.html#re-objects). This is the index into the string beyond which the RE engine will not go.

`Match.lastindex`[](https://docs.python.org/3/library/re.html#re.Match.lastindex "Permalink to this definition")
:    The integer index of the last matched capturing group, or&nbsp;`None`&nbsp;if no group was matched at all. For example, the expressions&nbsp;`(a)b`,&nbsp;`((a)(b))`, and&nbsp;`((ab))`&nbsp;will have&nbsp;`lastindex == 1`&nbsp;if applied to the string&nbsp;`'ab'`, while the expression&nbsp;`(a)(b)`&nbsp;will have&nbsp;`lastindex == 2`, if applied to the same string.

`Match.lastgroup`[](https://docs.python.org/3/library/re.html#re.Match.lastgroup "Permalink to this definition")
:    The name of the last matched capturing group, or&nbsp;`None`&nbsp;if the group didn’t have a name, or if no group was matched at all.

`Match.re`[](https://docs.python.org/3/library/re.html#re.Match.re "Permalink to this definition")
:    The&nbsp;[regular expression object](https://docs.python.org/3/library/re.html#re-objects)&nbsp;whose&nbsp;[`match()`](https://docs.python.org/3/library/re.html#re.Pattern.match "re.Pattern.match")&nbsp;or&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.Pattern.search "re.Pattern.search")&nbsp;method produced this match instance.

`Match.string`[](https://docs.python.org/3/library/re.html#re.Match.string "Permalink to this definition")
:    The string passed to&nbsp;[`match()`](https://docs.python.org/3/library/re.html#re.Pattern.match "re.Pattern.match")&nbsp;or&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.Pattern.search "re.Pattern.search").

Changed in version 3.7:&nbsp;Added support of&nbsp;[`copy.copy()`](https://docs.python.org/3/library/copy.html#copy.copy "copy.copy")&nbsp;and&nbsp;[`copy.deepcopy()`](https://docs.python.org/3/library/copy.html#copy.deepcopy "copy.deepcopy"). Match objects are considered atomic.

## Regular Expression Examples[](https://docs.python.org/3/library/re.html#regular-expression-examples "Permalink to this headline")

### Checking for a Pair[](https://docs.python.org/3/library/re.html#checking-for-a-pair "Permalink to this headline")

In this example, we’ll use the following helper function to display match objects a little more gracefully:

In [17]:
def displaymatch(match):
    if match is None:
        return None
    return '<Match: %r, groups=%r>' % (match.group(), match.groups())

Suppose you are writing a poker program where a player’s hand is represented as a 5-character string with each character representing a card, “a” for ace, “k” for king, “q” for queen, “j” for jack, “t” for 10, and “2” through “9” representing the card with that value.

To see if a given string is a valid hand, one could do the following:

In [18]:
valid = re.compile(r"^[a2-9tjqk]{5}$")
displaymatch(valid.match("akt5q"))  # Valid.

"<Match: 'akt5q', groups=()>"

In [19]:
displaymatch(valid.match("akt5e"))  # Invalid.
displaymatch(valid.match("akt"))    # Invalid.
displaymatch(valid.match("727ak"))  # Valid.

"<Match: '727ak', groups=()>"

That last hand,&nbsp;`"727ak"`, contained a pair, or two of the same valued cards. To match this with a regular expression, one could use backreferences as such:

In [20]:
pair = re.compile(r".*(.).*\1")
displaymatch(pair.match("717ak"))     # Pair of 7s.

"<Match: '717', groups=('7',)>"

In [21]:
displaymatch(pair.match("718ak"))     # No pairs.
displaymatch(pair.match("354aa"))     # Pair of aces.

"<Match: '354aa', groups=('a',)>"

To find out what card the pair consists of, one could use the&nbsp;[`group()`](https://docs.python.org/3/library/re.html#re.Match.group "re.Match.group")&nbsp;method of the match object in the following manner:

In [22]:
pair = re.compile(r".*(.).*\1")
pair.match("717ak").group(1)

'7'


Error because re.match() returns None, which doesn't have a group() method:


In [23]:
pair.match("718ak").group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [24]:
pair.match("354aa").group(1)

'a'

### Simulating scanf()[](https://docs.python.org/3/library/re.html#simulating-scanf "Permalink to this headline")

Python does not currently have an equivalent to&nbsp;`scanf()`. Regular expressions are generally more powerful, though also more verbose, than&nbsp;`scanf()`&nbsp;format strings. The table below offers some more-or-less equivalent mappings between&nbsp;`scanf()`&nbsp;format tokens and regular expressions.

| `scanf()`&nbsp;Token | Regular Expression |
|----------------------|--------------------|
| `%c` | `.` |
| `%5c` | `.{5}` |
| `%d` | `[-+]?\d+` |
| `%e`,&nbsp;`%E`,&nbsp;`%f`,&nbsp;`%g` | `[-+]?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?` |
| `%i` | `[-+]?(0[xX][\dA-Fa-f]+|0[0-7]*|\d+)` |
| `%o` | `[-+]?[0-7]+` |
| `%s` | `\S+` |
| `%u` | `\d+` |
| `%x`,&nbsp;`%X` | `[-+]?(0[xX])?[\dA-Fa-f]+` |

To extract the filename and numbers from a string like

/usr/sbin/sendmail - 0 errors, 4 warnings

you would use a&nbsp;`scanf()`&nbsp;format like

%s - %d errors, %d warnings

The equivalent regular expression would be

In [ ]:
(\S+) - (\d+) errors, (\d+) warnings

### search() vs. match()[](https://docs.python.org/3/library/re.html#search-vs-match "Permalink to this headline")

Python offers two different primitive operations based on regular expressions:&nbsp;[`re.match()`](https://docs.python.org/3/library/re.html#re.match "re.match")&nbsp;checks for a match only at the beginning of the string, while&nbsp;[`re.search()`](https://docs.python.org/3/library/re.html#re.search "re.search")&nbsp;checks for a match anywhere in the string (this is what Perl does by default).

For example:

In [25]:
re.match("c", "abcdef")    # No match
re.search("c", "abcdef")   # Match

<_sre.SRE_Match object; span=(2, 3), match='c'>

Regular expressions beginning with&nbsp;`'^'`&nbsp;can be used with&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.search "re.search")&nbsp;to restrict the match at the beginning of the string:


In [26]:
re.match("c", "abcdef")    # No match
re.search("^c", "abcdef")  # No match
re.search("^a", "abcdef")  # Match

<_sre.SRE_Match object; span=(0, 1), match='a'>

Note however that in&nbsp;[`MULTILINE`](https://docs.python.org/3/library/re.html#re.MULTILINE "re.MULTILINE")&nbsp;mode&nbsp;[`match()`](https://docs.python.org/3/library/re.html#re.match "re.match")&nbsp;only matches at the beginning of the string, whereas using&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.search "re.search")&nbsp;with a regular expression beginning with&nbsp;`'^'`&nbsp;will match at the beginning of each line.


In [27]:
re.match('X', 'A\nB\nX', re.MULTILINE)  # No match
re.search('^X', 'A\nB\nX', re.MULTILINE)  # Match

<_sre.SRE_Match object; span=(4, 5), match='X'>

### Making a Phonebook[](https://docs.python.org/3/library/re.html#making-a-phonebook "Permalink to this headline")

[`split()`](https://docs.python.org/3/library/re.html#re.split "re.split")&nbsp;splits a string into a list delimited by the passed pattern. The method is invaluable for converting textual data into data structures that can be easily read and modified by Python as demonstrated in the following example that creates a phonebook.

First, here is the input. Normally it may come from a file, here we are using triple-quoted string syntax

In [28]:
text = """Ross McFluff: 834.345.1254 155 Elm Street
Ronald Heathmore: 892.345.3428 436 Finley Avenue
Frank Burger: 925.541.7625 662 South Dogwood Way
Heather Albrecht: 548.326.4584 919 Park Place"""

The entries are separated by one or more newlines. Now we convert the string into a list with each nonempty line having its own entry:

In [29]:
entries = re.split("\n+", text)
entries

['Ross McFluff: 834.345.1254 155 Elm Street',
 'Ronald Heathmore: 892.345.3428 436 Finley Avenue',
 'Frank Burger: 925.541.7625 662 South Dogwood Way',
 'Heather Albrecht: 548.326.4584 919 Park Place']

Finally, split each entry into a list with first name, last name, telephone number, and address. We use the&nbsp;`maxsplit`&nbsp;parameter of&nbsp;[`split()`](https://docs.python.org/3/library/re.html#re.split "re.split")&nbsp;because the address has spaces, our splitting pattern, in it:

In [30]:
[re.split(":? ", entry, 3) for entry in entries]

[['Ross', 'McFluff', '834.345.1254', '155 Elm Street'],
 ['Ronald', 'Heathmore', '892.345.3428', '436 Finley Avenue'],
 ['Frank', 'Burger', '925.541.7625', '662 South Dogwood Way'],
 ['Heather', 'Albrecht', '548.326.4584', '919 Park Place']]

The&nbsp;`:?`&nbsp;pattern matches the colon after the last name, so that it does not occur in the result list. With a&nbsp;`maxsplit`of&nbsp;`4`, we could separate the house number from the street name:

In [31]:
[re.split(":? ", entry, 4) for entry in entries]

[['Ross', 'McFluff', '834.345.1254', '155', 'Elm Street'],
 ['Ronald', 'Heathmore', '892.345.3428', '436', 'Finley Avenue'],
 ['Frank', 'Burger', '925.541.7625', '662', 'South Dogwood Way'],
 ['Heather', 'Albrecht', '548.326.4584', '919', 'Park Place']]

### Text Munging[](https://docs.python.org/3/library/re.html#text-munging "Permalink to this headline")

[`sub()`](https://docs.python.org/3/library/re.html#re.sub "re.sub")&nbsp;replaces every occurrence of a pattern with a string or the result of a function. This example demonstrates using&nbsp;[`sub()`](https://docs.python.org/3/library/re.html#re.sub "re.sub")&nbsp;with a function to “munge” text, or randomize the order of all the characters in each word of a sentence except for the first and last characters:

In [33]:
import random
def repl(m):
    inner_word = list(m.group(2))
    random.shuffle(inner_word)
    return m.group(1) + "".join(inner_word) + m.group(3)
text = "Professor Abdolmalek, please report your absences promptly."
re.sub(r"(\w)(\w+)(\w)", repl, text)

'Posforesr Amaeoldlbk, pslaee rporet your ansbeecs ppotlmry.'

In [34]:
re.sub(r"(\w)(\w+)(\w)", repl, text)

'Psesofror Allmodbaek, psleae rrepot your aenebscs plmropty.'

### Finding all Adverbs[](https://docs.python.org/3/library/re.html#finding-all-adverbs "Permalink to this headline")

[`findall()`](https://docs.python.org/3/library/re.html#re.findall "re.findall")&nbsp;matches&nbsp;_all_&nbsp;occurrences of a pattern, not just the first one as&nbsp;[`search()`](https://docs.python.org/3/library/re.html#re.search "re.search")&nbsp;does. For example, if a writer wanted to find all of the adverbs in some text, they might use&nbsp;[`findall()`](https://docs.python.org/3/library/re.html#re.findall "re.findall")&nbsp;in the following manner:


In [35]:
text = "He was carefully disguised but captured quickly by police."
re.findall(r"\w+ly", text)

['carefully', 'quickly']

### Finding all Adverbs and their Positions[](https://docs.python.org/3/library/re.html#finding-all-adverbs-and-their-positions "Permalink to this headline")

If one wants more information about all matches of a pattern than the matched text,&nbsp;[`finditer()`](https://docs.python.org/3/library/re.html#re.finditer "re.finditer")&nbsp;is useful as it provides&nbsp;[match objects](https://docs.python.org/3/library/re.html#match-objects)&nbsp;instead of strings. Continuing with the previous example, if a writer wanted to find all of the adverbs&nbsp;_and their positions_&nbsp;in some text, they would use&nbsp;[`finditer()`](https://docs.python.org/3/library/re.html#re.finditer "re.finditer")&nbsp;in the following manner:

In [36]:
text = "He was carefully disguised but captured quickly by police."
for m in re.finditer(r"\w+ly", text):
    print('%02d-%02d: %s' % (m.start(), m.end(), m.group(0)))

07-16: carefully
40-47: quickly


### Raw String Notation[](https://docs.python.org/3/library/re.html#raw-string-notation "Permalink to this headline")

Raw string notation (`r"text"`) keeps regular expressions sane. Without it, every backslash (`'\'`) in a regular expression would have to be prefixed with another one to escape it. For example, the two following lines of code are functionally identical:

In [37]:
re.match(r"\W(.)\1\W", " ff ")

<_sre.SRE_Match object; span=(0, 4), match=' ff '>

In [38]:
re.match("\\W(.)\\1\\W", " ff ")

<_sre.SRE_Match object; span=(0, 4), match=' ff '>

When one wants to match a literal backslash, it must be escaped in the regular expression. With raw string notation, this means&nbsp;`r"\\"`. Without raw string notation, one must use&nbsp;`"\\\\"`, making the following lines of code functionally identical:


In [39]:
re.match(r"\\", r"\\")

<_sre.SRE_Match object; span=(0, 1), match='\\'>

In [40]:
re.match("\\\\", r"\\")

<_sre.SRE_Match object; span=(0, 1), match='\\'>

### Writing a Tokenizer[](https://docs.python.org/3/library/re.html#writing-a-tokenizer "Permalink to this headline")

A&nbsp;[tokenizer or scanner](https://en.wikipedia.org/wiki/Lexical_analysis)&nbsp;analyzes a string to categorize groups of characters. This is a useful first step in writing a compiler or interpreter.

The text categories are specified with regular expressions. The technique is to combine those into a single master regular expression and to loop over successive matches:

In [42]:
import collections
import re

Token = collections.namedtuple('Token', ['type', 'value', 'line', 'column'])

def tokenize(code):
    keywords = {'IF', 'THEN', 'ENDIF', 'FOR', 'NEXT', 'GOSUB', 'RETURN'}
    token_specification = [
        ('NUMBER',   r'\d+(\.\d*)?'),  # Integer or decimal number
        ('ASSIGN',   r':='),           # Assignment operator
        ('END',      r';'),            # Statement terminator
        ('ID',       r'[A-Za-z]+'),    # Identifiers
        ('OP',       r'[+\-*/]'),      # Arithmetic operators
        ('NEWLINE',  r'\n'),           # Line endings
        ('SKIP',     r'[ \t]+'),       # Skip over spaces and tabs
        ('MISMATCH', r'.'),            # Any other character
    ]
    tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    line_num = 1
    line_start = 0
    for mo in re.finditer(tok_regex, code):
        kind = mo.lastgroup
        value = mo.group()
        column = mo.start() - line_start
        if kind == 'NUMBER':
            value = float(value) if '.' in value else int(value)
        elif kind == 'ID' and value in keywords:
            kind = value
        elif kind == 'NEWLINE':
            line_start = mo.end()
            line_num += 1
            continue
        elif kind == 'SKIP':
            continue
        elif kind == 'MISMATCH':
            raise RuntimeError('{} unexpected on line {}'%(value,line_num))
        yield Token(kind, value, line_num, column)

statements = '''
    IF quantity THEN
        total := total + price * quantity;
        tax := price * 0.05;
    ENDIF;
'''

for token in tokenize(statements):
    print(token)

Token(type='IF', value='IF', line=2, column=4)
Token(type='ID', value='quantity', line=2, column=7)
Token(type='THEN', value='THEN', line=2, column=16)
Token(type='ID', value='total', line=3, column=8)
Token(type='ASSIGN', value=':=', line=3, column=14)
Token(type='ID', value='total', line=3, column=17)
Token(type='OP', value='+', line=3, column=23)
Token(type='ID', value='price', line=3, column=25)
Token(type='OP', value='*', line=3, column=31)
Token(type='ID', value='quantity', line=3, column=33)
Token(type='END', value=';', line=3, column=41)
Token(type='ID', value='tax', line=4, column=8)
Token(type='ASSIGN', value=':=', line=4, column=12)
Token(type='ID', value='price', line=4, column=15)
Token(type='OP', value='*', line=4, column=21)
Token(type='NUMBER', value=0.05, line=4, column=23)
Token(type='END', value=';', line=4, column=27)
Token(type='ENDIF', value='ENDIF', line=5, column=4)
Token(type='END', value=';', line=5, column=9)


The tokenizer produces the following output:

[Frie09](https://docs.python.org/3/library/re.html#id1)
:    Friedl, Jeffrey. Mastering Regular Expressions. 3rd ed., O’Reilly Media, 2009. The third edition of the book no longer covers Python at all, but the first edition covered writing good regular expression patterns in great detail.